# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
import json

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
output_data_file = "../WeatherPy/output/city_weather_data.csv"
city_df = pd.read_csv(output_data_file)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,277.15,87,20,4.63,CA,1623505834
1,Barrow,71.2906,-156.7887,272.16,86,90,3.60,US,1623505834
2,Centralia,46.7162,-122.9543,285.63,87,90,0.00,US,1623505834
3,Poum,-20.2333,164.0167,297.14,81,100,10.59,NC,1623505835
4,Cape Town,-33.9258,18.4232,294.88,54,0,0.89,ZA,1623505834


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
gmaps.configure(api_key=g_key)

In [27]:
locations = city_df[["Lat", "Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

![humidity_heatmap.png](humidity_heatmap.png)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
  # A max temperature lower than 275 degrees but higher than 250.
  # Wind speed less than 10 mph.
  # Zero cloudiness.
  # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


reduced_df = city_df.loc[(city_df["Max Temp"] < 275) & (city_df["Max Temp"] > 250) & (city_df["Wind Speed"] < 10), :]
reduced_df = reduced_df.dropna(how='any')
reduced_df.reset_index(inplace=True)
del reduced_df['index']

print(len(reduced_df.index))
reduced_df

6


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Barrow,71.2906,-156.7887,272.16,86,90,3.60,US,1623505834
1,Pangnirtung,66.1451,-65.7125,274.93,98,67,1.12,CA,1623505882
2,Leningradskiy,69.3833,178.4167,273.51,93,90,1.46,RU,1623505892
3,Dikson,73.5069,80.5464,273.33,89,65,1.78,RU,1623505897
4,Talnakh,69.4865,88.3972,274.42,99,100,3.16,RU,1623505915
5,Pevek,69.7008,170.3133,271.70,93,21,1.51,RU,1623505957


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = reduced_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Barrow,71.2906,-156.7887,272.16,86,90,3.60,US,1623505834,
1,Pangnirtung,66.1451,-65.7125,274.93,98,67,1.12,CA,1623505882,
2,Leningradskiy,69.3833,178.4167,273.51,93,90,1.46,RU,1623505892,
3,Dikson,73.5069,80.5464,273.33,89,65,1.78,RU,1623505897,
4,Talnakh,69.4865,88.3972,274.42,99,100,3.16,RU,1623505915,
5,Pevek,69.7008,170.3133,271.70,93,21,1.51,RU,1623505957,


In [8]:
for index, row in hotel_df.iterrows():
    try:
        
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        lat = row['Lat']
        lng = row['Lng']
        location = f"{lat}, {lng}"
     
        params = {
        "location": location,
        "keyword": "hotel",
        "radius": 8000,
        "key": g_key,
        }
        
        response = requests.get(url, params=params).json()
        print(json.dumps(response, indent=4, sort_keys=True))
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 71.2978529,
                    "lng": -156.7711574
                },
                "viewport": {
                    "northeast": {
                        "lat": 71.29892077989271,
                        "lng": -156.7695363701073
                    },
                    "southwest": {
                        "lat": 71.29622112010728,
                        "lng": -156.7722360298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Top of the World Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 500,
                    "html_attributions": [
                     

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 69.7041553,
                    "lng": 170.282609
                },
                "viewport": {
                    "northeast": {
                        "lat": 69.70572057989271,
                        "lng": 170.2838556798928
                    },
                    "southwest": {
                        "lat": 69.70302092010726,
                        "lng": 170.2811560201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Munits

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Barrow,71.2906,-156.7887,272.16,86,90,3.60,US,1623505834,Top of the World Hotel
1,Pangnirtung,66.1451,-65.7125,274.93,98,67,1.12,CA,1623505882,Auyuittuq Lodge (Inns North)
2,Leningradskiy,69.3833,178.4167,273.51,93,90,1.46,RU,1623505892,NaN
3,Dikson,73.5069,80.5464,273.33,89,65,1.78,RU,1623505897,NaN
4,Talnakh,69.4865,88.3972,274.42,99,100,3.16,RU,1623505915,NaN
5,Pevek,69.7008,170.3133,271.70,93,21,1.51,RU,1623505957,Munitsipal'naya Gostinitsa


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

![hotel_marker.png](hotel_marker.png)